<a href="https://colab.research.google.com/github/neirezcher/CodeAlpha-Task1-Recommendation_System/blob/main/Content_based_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install numpy pandas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/recommendationSystem/data/spotify_millsongdata.csv")

display(data)
data.info()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  57650 non-null  object
 1   song    57650 non-null  object
 2   link    57650 non-null  object
 3   text    57650 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [ ]:
data =data.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [ ]:
data.shape

(5000, 3)

Text preprocessing

In [ ]:
# convert text to lowercase and replace \n pattern by an empty space
data['text'] = data['text'].str.lower().replace(r'^\w\s', ' ', regex=True).replace(r'\n', ' ', regex=True).replace(r'\r', '', regex=True)

In [ ]:
display(data)

,artist,song,text
0,Kirsty Maccoll,Perfect Day,(lou reed) just a perfect day drink san...
1,Les Miserables,Do You Hear The People Sing,enjolras do you hear the people sing? sing...
2,Train,Drive By,on the other side of a street i knew stood a...
3,Frank Sinatra,Feelin' Kinda Sunday,"(feelin' kinda sunday, feelin' kinda sunday, f..."
4,One Direction,Kiss You,"[zayn] oh, i just wanna take you anywhere th..."
...,...,...,...
4995,Old 97's,Melt Show,in the front row at the melt show i fell in ...
4996,Hanson,Watch Over Me,she wakes and takes her place in line and ne...
4997,Ten Years After,Baby Won't You Let Me Rock'n Roll You,"baby, won't you let me rock 'n' roll you bab..."
4998,Doobie Brothers,It Keeps You Runnin',"say, where you gonna go girl, where you gonn..."


In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

# Ensure the necessary NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Initialize the stemmer
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
# Preprocess text: tokenize, remove stop words, and stem
def preprocess_text(txt):
    tokens = nltk.word_tokenize(txt.lower())  # Tokenize and convert to lowercase
    tokens = [word for word in tokens if word.isalnum()]  # Remove non-alphanumeric tokens
    tokens = [word for word in tokens if word not in stop_words]  # Remove stop words
    tokens = [stemmer.stem(word) for word in tokens]  # Apply stemming
    return tokens

# Assuming df is your DataFrame and it has a column named 'text'
data['preprocess_text'] = data['text'].apply(preprocess_text)

# Display the first few rows of the DataFrame to verify the changes
print(data.head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


           artist                         song  \
0  Kirsty Maccoll                  Perfect Day   
1  Les Miserables  Do You Hear The People Sing   
2           Train                     Drive By   
3   Frank Sinatra         Feelin' Kinda Sunday   
4   One Direction                     Kiss You   

                                                text  \
0  (lou reed)      just a perfect day   drink san...   
1  enjolras   do you hear the people sing?   sing...   
2  on the other side of a street i knew   stood a...   
3  (feelin' kinda sunday, feelin' kinda sunday, f...   
4  [zayn]   oh, i just wanna take you anywhere th...   

                                     preprocess_text  
0  [lou, reed, perfect, day, drink, sangria, park...  
1  [enjolra, hear, peopl, sing, sing, song, angri...  
2  [side, street, knew, stood, girl, look, like, ...  
3  [feelin, kinda, sunday, feelin, kinda, sunday,...  
4  [zayn, oh, wan, na, take, anywher, like, could...  


Using Embedded Vectors

In [ ]:
import gensim.downloader as api
# Load pre-trained Word2Vec model
model = api.load("word2vec-google-news-300")
# Function to average word vectors for a document
def document_vector(doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in model.key_to_index]
    if len(doc) == 0:
        return np.zeros(300)  # Return a zero vector if no words are in the vocabulary
    return np.mean(model[doc], axis=0)

# Apply vectorization to each document
data['vector'] = data['preprocess_text'].apply(document_vector)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# Create a matrix from the embedded vectors
matrix = np.stack(data['vector'].to_numpy())

# Calculate cosine similarity
similarity = cosine_similarity(matrix)

In [ ]:
print(similarity)

[[0.9999994  0.6709584  0.8021833  ... 0.7115479  0.7736024  0.71066225]
 [0.6709584  0.9999998  0.7111404  ... 0.5890281  0.6294576  0.6550337 ]
 [0.8021833  0.7111404  1.0000006  ... 0.74543035 0.837146   0.8327236 ]
 ...
 [0.7115479  0.5890281  0.74543035 ... 0.9999996  0.68783206 0.577456  ]
 [0.7736024  0.6294576  0.837146   ... 0.68783206 1.0000001  0.7048958 ]
 [0.71066225 0.6550337  0.8327236  ... 0.577456   0.7048958  1.        ]]


Recommender System

In [ ]:
data[data['song'] == 'Perfect Day']

,artist,song,text,preprocess_text,vector
0,Kirsty Maccoll,Perfect Day,(lou reed) just a perfect day drink san...,"[lou, reed, perfect, day, drink, sangria, park...","[0.026448568, 0.08586708, -0.018952198, 0.1107..."


In [ ]:
class Recommender:
    def __init__(self, data, similarity):
        self.df = data
        self.similarity_matrix = similarity


    def recommend(self, song_index, number_songs):
        if song_index >= len(self.similarity_matrix):
            raise IndexError("Song index out of range.")

        # Get the similarities for the given song
        similarities = self.similarity_matrix[song_index]

        # Get the indices of the most similar songs, excluding the song itself
        similar_indices = similarities.argsort()[::-1][1:number_songs+1]

        # Return the indices and similarities of the recommended songs
        return [(index, similarities[index]) for index in similar_indices]

    def _print_message(self, song_index, recom_song):
        song_name = self.df.iloc[song_index]['song']  # Assuming 'title' is the column with song titles
        print(f"Recommendations for {song_name}:")
        for idx, score in recom_song:
            print(f"Song: {self.df.iloc[idx]['song']}, Similarity Score: {score}")

In [ ]:
recommender = Recommender(data,similarity)

# Get recommendations
song_index = 0  # Index of the song for which recommendations are needed
number_songs = 3  # Number of recommendations needed

try:
    recommendations = recommender.recommend(song_index, number_songs)
    recommender._print_message(song_index, recommendations)
except IndexError as e:
    print(f"Error: {e}")

Recommendations for Perfect Day:
Song: Forget About Us, Similarity Score: 0.8581979870796204
Song: Danny's Song, Similarity Score: 0.8544533252716064
Song: Save Me From Myself, Similarity Score: 0.8542169332504272
